In [37]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [47]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.iloc[:1170]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
df.set_index('Date', inplace=True)
df

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-3.77,-4.31,-2.37,1.95,-1.68,0.14,-0.40,-0.22,-5.29,-3.35
2023-09-01,-4.57,-2.58,-7.08,3.17,-5.98,-3.22,-5.68,-4.71,-5.04,-3.41
2023-10-01,-3.53,-17.88,-2.76,-6.24,-1.78,-0.18,0.47,-4.58,1.12,-2.53


In [12]:
window_size = 60
rolling_window = df.rolling(window=window_size)

In [29]:
# Calculer la moyenne des rendements pour chaque fenêtre
rolling_mean = df.rolling(window=window_size).mean().dropna().values/100

# Calculer la covariance des rendements pour chaque fenêtre
rolling_covariance = df.rolling.cov(window=window_size).cov().dropna().values/100

# Afficher la moyenne et la covariance pour chaque fenêtre
rolling_mean
rolling_covariance

array([[ 0.0064    ,  0.00985167,  0.00718   , ...,  0.01102667,
         0.01756333,  0.00162167],
       [ 0.00552333,  0.00691   ,  0.00469333, ...,  0.01116667,
         0.01549167, -0.00037667],
       [ 0.00475333,  0.00583333,  0.0042    , ...,  0.01052333,
         0.01548333, -0.001765  ],
       ...,
       [ 0.00495667,  0.02645333,  0.008395  , ...,  0.00644667,
         0.00554833,  0.00815333],
       [ 0.00571333,  0.02819167,  0.00906833, ...,  0.006355  ,
         0.00584833,  0.00942   ],
       [ 0.00818333,  0.03118833,  0.01158333, ...,  0.00888   ,
         0.00692333,  0.01233333]])

In [ ]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()